In [2]:
import pandas as pd
import re
from time import sleep
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [3]:
# Interprération d'une date en format "YYYYMMDD"
def parse_date(_date):
    annee = _date[0:4]
    mois = _date[4:6]
    jour = _date[6:8]
    if mois == '00':
        mois = '07'
    if jour == '00':
        jour = '01'
    if annee == '0000':
        return None
    return datetime(int(annee), int(mois), int(jour))

In [4]:
# Calcul de l'age révolu à la date du décès
def age_revolu(date_naissance, date_deces):
    _date_naissance = parse_date(date_naissance)
    _date_deces = parse_date(date_deces)
    if _date_naissance and _date_deces:
        _age_revolu = int((_date_deces - _date_naissance).days/365)
        return _age_revolu, _date_naissance, _date_deces
    else:
        return None, None, None

In [5]:
# Expression régulière pour interpréter une ligne du fichier de décès INSEE
_regexp = r'([A-Z\s\-\'\*]+)\/\s*([12])([0-9]{8})([0-9AB]{5})(.+[^0-9])([0-9]{8})([0-9AB]{5})\s*([0-9]*)'
_pattern = re.compile(_regexp)

In [6]:
def parse(_string, annee_fichier):
    _match = _pattern.match(_string)
    if _match:
        _age_revolu, _date_naissance, _date_deces = age_revolu(_match[3], _match[6])
        return {'NOM_PRENOMS': _match[1],
                'SEXE': "H" if _match[2] == "1" else "F",
                'DATE_NAISSANCE': _date_naissance,
                'COMM_NAISSANCE': _match[4],
                'LIEU_NAISSANCE': _match[5],
                'DATE_DECES': _date_deces,
                'COMM_DECES': _match[7],
                'ID': _match[8],
                'AGE_REVOLU': _age_revolu,
                'ANNEE': int(_match[6][0:4]),
                'ANNEE_FICHIER': annee_fichier
               }

In [7]:
# Exemple
parse('OSWALD*ANNA/                                                                    21928030599109LORRACH                       ALLEMAGNE                     201912071410020                ', 2020)

{'AGE_REVOLU': 91,
 'ANNEE': 2019,
 'ANNEE_FICHIER': 2020,
 'COMM_DECES': '14100',
 'COMM_NAISSANCE': '99109',
 'DATE_DECES': datetime.datetime(2019, 12, 7, 0, 0),
 'DATE_NAISSANCE': datetime.datetime(1928, 3, 5, 0, 0),
 'ID': '20',
 'LIEU_NAISSANCE': 'LORRACH                       ALLEMAGNE                     ',
 'NOM_PRENOMS': 'OSWALD*ANNA',
 'SEXE': 'F'}

In [8]:
fichiers_insee_deces = {
    2020: 'https://www.data.gouv.fr/fr/datasets/r/a1f09595-0e79-4300-be1a-c97964e55f05',
    2019: 'https://www.data.gouv.fr/fr/datasets/r/02acf8f5-9190-4f8e-a37c-3b34eccac833',
    2018: 'https://www.data.gouv.fr/fr/datasets/r/c2a97b38-5c0d-4f21-910f-1cea164c2c89',
    2017: 'https://www.data.gouv.fr/fr/datasets/r/fd61ff96-1e4e-450f-8648-3e3016edbe34',
    2016: 'https://www.data.gouv.fr/fr/datasets/r/8fb032c1-b81e-46c4-a48a-15380ce41e40',
}

In [9]:
# Récupration et lecture des fichiers de décès INSEE
_dataframes = {annee_fichier: pd.read_csv(_fichier, sep='\t', header=None, encoding='latin1')
               for annee_fichier, _fichier in fichiers_insee_deces.items()}

In [19]:
# Concatenation et elimination des quelques lignes dont les informations sont inexploitables
deces = pd.concat([pd.DataFrame([x for x in df[0].map(lambda x: parse(x, annee_fichier)) if x])
                   for annee_fichier, df in _dataframes.items()])
deces = deces.query('AGE_REVOLU==AGE_REVOLU and SEXE==SEXE and ANNEE==ANNEE')

In [189]:
['Fichier {} : {} décès dont {}% en France métropolitaine et {} inexploitables'.format
 (annee_fichier,
  k_format(df.shape[0]),
  int(10000*deces[~deces['COMM_DECES'].str.startswith('99')].query(f'ANNEE_FICHIER=={annee_fichier}').shape[0]/df.shape[0])/100,
  df.shape[0] - deces.query(f'ANNEE_FICHIER=={annee_fichier}').shape[0],
)
for annee_fichier, df in _dataframes.items()]

['Fichier 2020 : 679.9k décès dont 99.1% en France métropolitaine et 3 inexploitables',
 'Fichier 2019 : 625.3k décès dont 98.88% en France métropolitaine et 2 inexploitables',
 'Fichier 2018 : 620.1k décès dont 98.8% en France métropolitaine et 5 inexploitables',
 'Fichier 2017 : 612.9k décès dont 98.74% en France métropolitaine et 5 inexploitables',
 'Fichier 2016 : 603.3k décès dont 98.67% en France métropolitaine et 4 inexploitables']

In [187]:
deces['DATE_DECES'].map(lambda _date: (_date + timedelta(days=31)).year).value_counts().map(k_format)[0:5]

2020    665.2k
2018    622.6k
2019    619.5k
2017    615.9k
2016    555.7k
Name: DATE_DECES, dtype: object

In [ ]:
deces.to_csv('deces.csv.gz', compression='gzip')

In [32]:
# Limitation au décès survenus en France metropolitaine
deces_metro = deces[~deces['COMM_DECES'].str.startswith('99')]
# Suppression des données du 29 Février 2020, seule année bissextile
deces_metro = deces_metro[deces_metro['DATE_DECES'] != '20200229']

In [50]:
# 99.5% des décès metropolitains hors décembre d'une année sont remontés dans le fichier de la même année
for annee in [2019, 2018, 2017, 2016]:
    deces_annee_n = deces_metro.query(f'DATE_DECES>="{annee}-01-01" and DATE_DECES<"{annee}-12-31"')
    fichier_n_plus = deces_annee_n.query(f'ANNEE_FICHIER=={annee+1}')['DATE_DECES'].count()
    fichier_n_plus += deces_annee_n.query(f'ANNEE_FICHIER=={annee+2}')['DATE_DECES'].count()
    fichier_n_plus += deces_annee_n.query(f'ANNEE_FICHIER=={annee+3}')['DATE_DECES'].count()
    print(annee, int(10000*(1 - fichier_n_plus/deces_annee_n.shape[0]))/100)

2019 98.08
2018 97.74
2017 97.63
2016 98.11


In [51]:
# 99.5% des décès metropolitains hors décembre d'une année sont remontés dans le fichier de la même année
for annee in [2019, 2018, 2017, 2016]:
    deces_annee_n_hors_decembre = deces_metro.query(f'DATE_DECES>="{annee}-01-01" and DATE_DECES<"{annee}-12-01"')
    fichier_n_plus = deces_annee_n_hors_decembre.query(f'ANNEE_FICHIER=={annee+1}')['DATE_DECES'].count()
    fichier_n_plus += deces_annee_n_hors_decembre.query(f'ANNEE_FICHIER=={annee+2}')['DATE_DECES'].count()
    fichier_n_plus += deces_annee_n_hors_decembre.query(f'ANNEE_FICHIER=={annee+3}')['DATE_DECES'].count()
    print(annee, int(10000*(1 - fichier_n_plus/deces_annee_n_hors_decembre.shape[0]))/100)

2019 99.82
2018 99.72
2017 99.59
2016 99.74


In [39]:
# Re-definition de l'année de décès comme l'année de la (date de deces + 31 jours), afin de capturer au moins 99.5% des décès par année glissante
deces_metro['ANNEE'] = deces_metro['DATE_DECES'].map(lambda _date: (_date + timedelta(days=31)).year)

In [192]:
deces_metro.groupby(['ANNEE', 'SEXE'])['NOM_PRENOMS'].count().map(k_format).reset_index().query('ANNEE>=2017 and ANNEE<=2020')

,ANNEE,SEXE,NOM_PRENOMS
166,2017,F,306.4k
167,2017,H,302.0k
168,2018,F,309.1k
169,2018,H,306.8k
170,2019,F,308.1k
171,2019,H,305.3k
172,2020,F,328.9k
173,2020,H,330.3k


In [52]:
# Pyramide des ages par sexe en France métropolitaine, celle de 2016 n'est pas facilement disponible
# Attention, le site de l'INSEE connaît de temps en temps des latences ou indisponibilités
pyramides_metro = {}
for annee in [2017, 2018, 2019, 2020]:
  print(annee)
  pyramides_metro[annee] = pd.read_excel(f'https://www.insee.fr/fr/statistiques/fichier/1913143/pyramide-des-ages-{annee}.xls', sheet_name=f'{annee} Métro')[5:-2]
  sleep(5)

2017
2018
2019
2020


In [53]:
# Quelques reformatages
for annee, df in pyramides_metro.items():
    df.columns = ['ANNEE_NAISSANCE', 'AGE_REVOLU', 'NB_H', 'NB_F', 'NB']
    df['ANNEE'] = annee
    df['AGE_REVOLU'] = df['AGE_REVOLU'].astype(int)

In [54]:
# Concaténation pour avoir un seul dataframe
pyramide_metro_orig = pd.concat(pyramides_metro.values())

In [55]:
# Reformatage pour avoir une ligne par année, age révolu et sexe
df_h = pyramide_metro_orig[['ANNEE', 'AGE_REVOLU', 'NB_H']].rename(columns={'NB_H': 'NB'})
df_h['SEXE'] = "H"
df_f = pyramide_metro_orig[['ANNEE', 'AGE_REVOLU', 'NB_F']].rename(columns={'NB_F': 'NB'})
df_f['SEXE'] = "F"
pyramide_metro = pd.concat([df_h, df_f])

In [56]:
# Aggrégation des décès par année, age révolu (jusqu'à 96 ans) et sexe
deces_metro_agg = deces_metro.groupby(['AGE_REVOLU', 'ANNEE', 'SEXE'])['DATE_DECES'].count().reset_index()
deces_metro_agg = deces_metro_agg.query('AGE_REVOLU < 97')
deces_metro_agg.columns = ['AGE_REVOLU', 'ANNEE', 'SEXE', 'NB_DECES']

In [76]:
# Jointure entre les pyrmides d'âge et les décès par age révolu
_merge = deces_metro_agg.merge(pyramide_metro)

In [77]:
# Retourne les taux de mortalité par age révolu pour une année et un sexe donnés
def mortalite_par_age(df, annee, sexe):
    _df = df[(df['ANNEE'] == annee) & (df['SEXE']==sexe)]
    _df = _df.set_index(_df['AGE_REVOLU'])
    return pd.DataFrame(_df['NB_DECES'] / _df['NB'], columns=[f'{annee}_{sexe}_MORTALITE'])

In [78]:
# Calcul des series de taux de mortalité pour toutes les combinaisons d'année et sexe
taux_mortalite = pd.concat([mortalite_par_age(_merge, annee, sexe)
                            for annee in [2020, 2019, 2018, 2017]
                            for sexe in ["H", "F"]], axis=1)

In [184]:
# Enrichissement avec la pyramide d'age 2020 et la mortalité max et min observée entre 2017 et 2019
for sexe in ["H", "F"]:
    taux_mortalite[f'NB_2020_{sexe}'] = _merge.query(f'SEXE=="{sexe}" and ANNEE==2020').set_index('AGE_REVOLU')['NB']
    taux_mortalite[f'MAX_{sexe}_MORTALITE'] =taux_mortalite[[f'{annee}_{sexe}_MORTALITE'
                                                             for annee in [2017, 2018, 2019]]].max(axis=1)
    taux_mortalite[f'MIN_{sexe}_MORTALITE'] = taux_mortalite[[f'{annee}_{sexe}_MORTALITE'
                                                              for annee in [2017, 2018, 2019]]].min(axis=1)
    taux_mortalite[f'MOY_{sexe}_MORTALITE'] = taux_mortalite[[f'{annee}_{sexe}_MORTALITE'
                                                              for annee in [2017, 2018, 2019]]].mean(axis=1)

In [158]:
def k_format(_float):
  return f'{int(_float/100)/10}k'

In [182]:
_list = []
for sexe in ["F", "H"]:
  for ages in [(0, 44), (45, 64), (65, 74), (75, 89), (90, 96)]:
    _pop = taux_mortalite.query(f'AGE_REVOLU>={ages[0]} and AGE_REVOLU<={ages[1]}')[f'NB_2020_{sexe}']
    _deces_2020 = _pop.multiply(taux_mortalite[f'2020_{sexe}_MORTALITE']).sum()
    _enveloppe_min = _pop.multiply(taux_mortalite[f'MIN_{sexe}_MORTALITE']).sum()
    _enveloppe_moy = _pop.multiply(taux_mortalite[f'MOY_{sexe}_MORTALITE']).sum()
    _enveloppe_max = _pop.multiply(taux_mortalite[f'MAX_{sexe}_MORTALITE']).sum()
    _depassement = _deces_2020 - _enveloppe_max
    _ecart_moy = _deces_2020 - _enveloppe_moy
    _list.append(
        {'SEXE': 'Hommes' if sexe=="H" else 'Femmes', 'AGE_REVOLU': f'{ages[0]} à {ages[1]}',
         'DECES_2020': f'{k_format(_deces_2020)}',
         'ENVELOPPE_ATTENDUE': f'{k_format(_enveloppe_min)} à {k_format(_enveloppe_max)}',
         'ECART_MOYENNE_ATTENDUE':  f'{k_format(_ecart_moy)}',
         'DEPASSEMENT': f'{k_format(_depassement)}'
         })

In [183]:
pd.DataFrame(_list)

,SEXE,AGE_REVOLU,DECES_2020,ENVELOPPE_ATTENDUE,ECART_MOYENNE_ATTENDUE,DEPASSEMENT
0,Femmes,0 à 44,6.5k,6.3k à 7.2k,-0.2k,-0.7k
1,Femmes,45 à 64,27.8k,27.2k à 28.4k,0.0k,-0.6k
2,Femmes,65 à 74,36.6k,34.3k à 36.9k,1.1k,-0.3k
3,Femmes,75 à 89,130.6k,123.1k à 130.8k,3.5k,-0.2k
4,Femmes,90 à 96,97.2k,92.1k à 98.4k,1.8k,-1.2k
5,Hommes,0 à 44,12.7k,12.5k à 13.7k,-0.4k,-1.0k
6,Hommes,45 à 64,52.8k,51.6k à 54.4k,-0.2k,-1.5k
7,Hommes,65 à 74,66.7k,62.1k à 66.9k,2.7k,-0.1k
8,Hommes,75 à 89,139.2k,128.0k à 136.3k,6.7k,2.8k
9,Hommes,90 à 96,50.4k,47.0k à 48.9k,2.4k,1.5k


In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.update_layout(
    title='Taux de mortalité par âge révolu',
    font=dict(
        family="Century Gothic",
        size=18,
    ),
    legend=dict(
      yanchor="top", y=0.99, xanchor="left", x=0.01
    )
)

In [97]:
sexe = 'H'
col = f'MIN_{sexe}_MORTALITE'
fig.add_trace(go.Scatter(x=taux_mortalite[col].index, y=taux_mortalite[col], name=col, line_color='grey'))
col = f'MAX_{sexe}_MORTALITE'
fig.add_trace(go.Scatter(x=taux_mortalite[col].index, y=taux_mortalite[col], name=col, line_color='grey', fill='tonexty'))
col = f'2020_{sexe}_MORTALITE'
fig.add_trace(go.Scatter(x=taux_mortalite[col].index, y=taux_mortalite[col], name=col, line_color='red'))
fig.show()